In [42]:
import pandas as pd
import numpy as np
from scipy.special import ndtr


In [43]:

df = pd.read_excel('df_fig7_data.xlsx')
df = df.rename(columns={'Fuel Type': 'FuelType'})
df


,date,year,month,hour,Load,mkwh,FuelType,Fuel
0,2019-01-01,2019,1,0,461392,109616,0,Coal
1,2019-01-01,2019,1,1,459577,108132,0,Coal
2,2019-01-01,2019,1,2,451601,103544,0,Coal
3,2019-01-01,2019,1,3,437803,98623,0,Coal
4,2019-01-01,2019,1,4,422742,95047,0,Coal
...,...,...,...,...,...,...,...,...
122731,2025-12-31,2025,12,19,507946,166513,1,Gas
122732,2025-12-31,2025,12,20,502367,164730,1,Gas
122733,2025-12-31,2025,12,21,498110,166528,1,Gas
122734,2025-12-31,2025,12,22,503223,175617,1,Gas


In [44]:
# Structural break
df['may22'] = (df['year'] > 2021).astype(int)
df.loc[(df['year']==2022) & (df['month']<=4), 'may22'] = 0

# Interaction terms
df['load_may22'] = df['Load'] * df['may22']
df['may22_FuelType'] = df['may22'] * df['FuelType']
df['load_may22_FuelType'] = df['Load'] * df['may22'] * df['FuelType']
# Full 3-way FE interaction group (year x month x hour)
df['FE_ID'] = df['year'].astype(str) + "_" + df['month'].astype(str) + "_" + df['hour'].astype(str)


In [53]:
import statsmodels.formula.api as smf

formula = 'mkwh ~ Load * may22 * C(FuelType)' 

ddd_model = smf.ols(formula, data=df).fit(cov_type='cluster', cov_kwds={'groups': df['FE_ID']})

print("\n--- Triple Difference Results ---")
print(ddd_model.summary())


--- Triple Difference Results ---
                            OLS Regression Results                            
Dep. Variable:                   mkwh   R-squared:                       0.930
Model:                            OLS   Adj. R-squared:                  0.930
Method:                 Least Squares   F-statistic:                 1.670e+04
Date:                gio, 19 feb 2026   Prob (F-statistic):               0.00
Time:                        15:35:07   Log-Likelihood:            -1.3575e+06
No. Observations:              122736   AIC:                         2.715e+06
Df Residuals:                  122728   BIC:                         2.715e+06
Df Model:                           7                                         
Covariance Type:              cluster                                         
                                  coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------------------------

In [50]:
from linearmodels import PanelOLS

# 1. Ensure date is in datetime format
df['date'] = pd.to_datetime(df['date'])

# 2. Set the Multi-Index 
# First level is the group to be absorbed (FE_ID)
df_panel = df.set_index(['FE_ID', 'date'])

# 3. Define the Triple Difference Model
from linearmodels import PanelOLS

# 1. Define the model as before
model = PanelOLS.from_formula(
    'mkwh ~ Load * may22 * C(FuelType) + EntityEffects', 
    data=df_panel,
    drop_absorbed=True  
)

# 2. Fit the model
# Using 'clustered' matches the HDFE robust standard errors
results = model.fit(cov_type='clustered', cluster_entity=True)
#results = model.fit(cov_type='unadjusted', auto_df=True)
print("\n--- Triple Difference Results (PanelOLS) ---")
print(results.summary.tables[1])


--- Triple Difference Results (PanelOLS) ---
                                      Parameter Estimates                                      
                             Parameter  Std. Err.     T-stat    P-value    Lower CI    Upper CI
-----------------------------------------------------------------------------------------------
Load                            0.3515     0.0047     74.382     0.0000      0.3423      0.3608
may22                       -1.701e+05     5451.0    -31.198     0.0000  -1.807e+05  -1.594e+05
C(FuelType)[0]               3.501e+04     3553.9     9.8527     0.0000   2.805e+04   4.198e+04
Load:may22                     -0.0757     0.0059    -12.898     0.0000     -0.0872     -0.0642
Load:C(FuelType)[T.1]           0.2225     0.0079     28.064     0.0000      0.2070      0.2380
may22:C(FuelType)[T.1]          3218.0     4556.4     0.7063     0.4800     -5712.4   1.215e+04
Load:may22:C(FuelType)[T.1]     0.0722     0.0099     7.3074     0.0000      0.0528      0

C:\Users\Antonio\AppData\Local\Temp\ipykernel_114104\232487391.py:22: AbsorbingEffectWarning: 
Variables have been fully absorbed and have removed from the regression:

C(FuelType)[1]

  results = model.fit(cov_type='clustered', cluster_entity=True)
